In [1]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

KeyboardInterrupt: 

In [1]:

# Define the directory paths
normal_dir = 'chest_xray/train/NORMAL'
pneumonia_dir = 'chest_xray/train/PNEUMONIA'

# Get the list of all files in each directory
normal_files = [os.path.join(normal_dir, file) for file in os.listdir(normal_dir)]
pneumonia_files = [os.path.join(pneumonia_dir, file) for file in os.listdir(pneumonia_dir)]

# Create a DataFrame
df = pd.DataFrame({
    'file': normal_files + pneumonia_files,
    'label': ['normal'] * len(normal_files) + ['pneumonia'] * len(pneumonia_files)
})

print(df)

                                                   file      label
0             chest_xray/train/NORMAL\IM-0115-0001.jpeg     normal
1             chest_xray/train/NORMAL\IM-0117-0001.jpeg     normal
2             chest_xray/train/NORMAL\IM-0119-0001.jpeg     normal
3             chest_xray/train/NORMAL\IM-0122-0001.jpeg     normal
4             chest_xray/train/NORMAL\IM-0125-0001.jpeg     normal
...                                                 ...        ...
3787  chest_xray/train/PNEUMONIA\person438_bacteria_...  pneumonia
3788  chest_xray/train/PNEUMONIA\person438_bacteria_...  pneumonia
3789  chest_xray/train/PNEUMONIA\person438_bacteria_...  pneumonia
3790  chest_xray/train/PNEUMONIA\person438_bacteria_...  pneumonia
3791  chest_xray/train/PNEUMONIA\person438_bacteria_...  pneumonia

[3792 rows x 2 columns]


In [ ]:
# Initialize the ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255.,  # scale pixel values to [0, 1]
    shear_range=0.2,  # randomly apply shearing transformations
    zoom_range=0.2,  # randomly zooming inside pictures
    horizontal_flip=True  # randomly flip half of the images horizontally
)